In [ ]:
import os
import pandas as pd
import numpy as np
from twitter.scraper import *
from IPython.display import display, HTML
import re, pandas as pd, numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, OpenAI
from sentence_transformers import SentenceTransformer
import openai
from pprint import pprint

from tqdm.notebook import tqdm as notebook_tqdm
# !pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu129

pd.set_option("display.max_colwidth", None)  # show full text in each cell
print(torch.cuda.is_available())
print(torch.version.cuda)

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
roberta_model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
# sentiment_task = pipeline("sentiment-analysis", model=roberta_model_path, tokenizer=roberta_model_path)
tokenizer = AutoTokenizer.from_pretrained(roberta_model_path, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(roberta_model_path)
model.to('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
pass

In [ ]:
import importlib
import twitter
importlib.reload(twitter)
from twitter import *
from datetime import datetime

def relevancy_score(tweet, decay=0.7):
    # decay=1.0: linear penalty → each extra day reduces score proportionally.
    # Smaller decay (e.g. 0.5) → recency matters less.
    # Larger decay (e.g. 2) → recency matters more strongly.
    created_at = datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S %z %Y")
    created_at = created_at.replace(tzinfo=None)  # drop tz for consistency
    age_days = (datetime.now() - created_at).days + 1   # +1 to avoid div by 0
    return tweet["replies"] / (age_days ** decay)

def clean_tweet(t):
    t = re.sub(r"http\S+", "", t)  
    t = re.sub(r"\s+", " ", t).strip()
    if t.startswith('@'):
        parts = t.split(" ", 1)
        if len(parts) > 1:
            return parts[1]
        else:
            return t
    return t
    
def stringify_tweet(tweet: Tweet):
    full_tweet = []
    cleaned_tweet_text = clean_tweet(tweet["text"])
    if cleaned_tweet_text: 
        full_tweet.append(cleaned_tweet_text)
    if tweet["post_image_description"]:
        full_tweet.append(tweet["post_image_description"])
    if tweet["post_video_description"]:
        full_tweet.append(tweet["post_video_description"])
    
    if tweet["quoted_tweet"]:
        full_tweet+= [stringify_tweet(tweet["quoted_tweet"])]
    return "\n".join(full_tweet)


rocket_lab = '91145174'
peter_beck = '976574172468936704'
nvidia= '61559439'
elon_musk = '44196397'

tweets = get_user_tweets(rocket_lab, period='days=15')[0]

top_20_tweets = sorted(tweets, key=lambda x: relevancy_score(x), reverse=True)[:20]
comments = get_comments('1959432404080230691', minimum_comments=200)[0]
# comments = get_comments(top_20_tweets[0]["tweet_id"], minimum_comments=200)[0]
original_tweet = comments[0]
comments = comments[1:]


In [ ]:
cleaned_tweets = []
cleaned_comments = []
clean_comments_obj = []

print(f'Tweet count: {len(tweets)}')
print("Newest Tweet Date: ", tweets[0]["created_at"])
print("Oldest Tweet Date: ", tweets[-1]["created_at"])

for tweet in tweets:
    if tweet["retweeted_tweet"]: continue
    full_tweet = stringify_tweet(tweet)
    if not full_tweet: continue
    cleaned_tweets.append(full_tweet)
    # print(full_tweet)
    # print("-"*100)

print(f"Cleaned tweet count: {len(cleaned_tweets)}/{len(tweets)}")
print('='*100)

# print(top_20_tweets[0])
for comment in comments[1:]: # first comment is the original tweet
    cleaned_comment = stringify_tweet(comment)
    if not cleaned_comment: continue
    cleaned_comments.append(cleaned_comment)
    clean_comments_obj.append(comment)
    print(cleaned_comment)
    print("-"*100)

print("Oldest Comment Date: ", comments[0]["created_at"])
print("Newest Comment Date: ", comments[-1]["created_at"])
print(f"{len(cleaned_comments)}/{len(comments[1:])}")
print(len(clean_comments_obj))

In [ ]:
def rob_score_from_texts(texts: List[str], batch_size: int = 32) -> List[float]:
    """
    Batch inference returning continuous scores in [-1, 1]:
    score = prob_pos - prob_neg
    For models with 3 labels arranged [negative, neutral, positive].
    """
    scores = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i : i + batch_size]
            enc = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=256)
            enc = {k: v.to('cuda') for k, v in enc.items()}
            out = model(**enc)
            logits = out.logits.cpu()
            probs = torch.softmax(logits, dim=-1).numpy()  # shape (batch, num_labels)
            # mapping - assumes label order is neg, neu, pos
            # if model uses different label order, you must reorder accordingly
            for p in probs:
                if p.shape[0] == 3:
                    prob_neg, prob_neu, prob_pos = p
                    score = float(prob_pos - prob_neg)  # in (-1, 1)
                elif p.shape[0] == 2:
                    # binary model mapping (pos prob - neg prob)
                    prob_neg = p[0]
                    prob_pos = p[1]
                    score = float(prob_pos - prob_neg)
                else:
                    # fallback: compute (expected label index scaled)
                    labels = np.arange(len(p))
                    norm = (labels - labels.mean()) / (labels.max() - labels.min() + 1e-9)
                    score = float(np.dot(p, norm))
                    # then scale to -1..1
                    score = max(-1.0, min(1.0, score))
                scores.append(score)
    return scores

def combine_scores(roberta_scores, vader_scores, roberta_weight=0.7, vader_weight=0.3):
    """
    Vectorized combination of RoBERTa and VADER sentiment scores (both in [-1, 1]).
    - If models strongly disagree (|diff|>0.6) or VADER is 0, trust RoBERTa.
    - When signs disagree, upweight RoBERTa (0.9/0.1) and apply disagreement damping.
    Returns a list of combined scores in [-1, 1].
    """
    sR = np.asarray(roberta_scores, dtype=float)
    sV = np.asarray(vader_scores, dtype=float)
    wR = np.full_like(sR, float(roberta_weight))
    wV = np.full_like(sR, float(vader_weight))

    diff = np.abs(sR - sV)
    strong_mask = (diff > 0.6) | (sV == 0.0) # strong disagreement or VADER is exactly neutral → trust RoBERTa
    
    # if disagree, upweight RoBERTa and dampen
    sign_disagree = (sR * sV) < 0.0 
    wR = np.where(sign_disagree, 0.9, wR)
    wV = np.where(sign_disagree, 0.1, wV)
    damp_factor = np.ones_like(sR, dtype=float)
    damp_factor = np.where(sign_disagree, 1.0 - 0.5 * np.minimum(diff, 1.0), damp_factor)

    combined = (wR * sR + wV * sV) / (wR + wV)
    combined = combined * damp_factor
    combined = np.where(strong_mask, sR, combined)

    combined = np.clip(combined, -1.0, 1.0)
    return combined.tolist()

vader_sentiments = [analyzer.polarity_scores(sentence)["compound"] for sentence in cleaned_comments]
roberta_sentiments = rob_score_from_texts(cleaned_comments)
combined_scores = combine_scores(roberta_sentiments, vader_sentiments)

In [ ]:
sentiment_df = pd.DataFrame()
sentiment_df["comment"] = cleaned_comments
sentiment_df["likes"] = [tweet["likes"] for tweet in clean_comments_obj]
sentiment_df["roberta"] = roberta_sentiments
sentiment_df["vader"] = vader_sentiments
sentiment_df["combined"] = combined_scores
sentiment_df['combined'] = (sentiment_df['combined'] + 1) / 2
eps = 1e-9
weighted_likes = np.log1p(sentiment_df['likes']) + eps
weighted_normalised_likes = weighted_likes / np.linalg.norm(weighted_likes)
sentiment_df['weighted_likes'] = weighted_normalised_likes
    
    

print(f"Overall sentiment: {np.average(sentiment_df['combined'], weights=weighted_normalised_likes)}")
display(HTML(sentiment_df.to_html(index=False)))


In [ ]:
roberta_df = pd.DataFrame()
sign_map = {"positive": 1.0, "neutral": 0.0, "negative": -1.0}
labels = {"negative": "❌", "neutral": "😐", "positive": "✅"}
roberta_df["label"] = [labels[sentiment["label"]] for sentiment in roberta_sentiments]
roberta_df["sentiment"] = [sign_map[sentiment["label"]] for sentiment in roberta_sentiments]
roberta_df["confidence"] = [sentiment["score"] for sentiment in roberta_sentiments]
roberta_df["comment"] = cleaned_comments
roberta_df["likes"] = [c["likes"] for c in clean_comments_obj]

likes = roberta_df["likes"].astype(float)
M = np.log1p(likes.max()) if likes.max() > 0 else 1.0
norm_log = np.log1p(likes) / M
baseline = 0.3
roberta_df["weight_likes"] = baseline + (1 - baseline) * norm_log

# per-row polarity in [-1, 1]
roberta_df["polarity"] = roberta_df["sentiment"] * roberta_df["confidence"]

# overall weighted sentiment in [-1, 1]
overall = np.average(roberta_df["polarity"], weights=roberta_df["weight_likes"])
print("Overall weighted sentiment (−1..1):", overall)
print("As % positive tilt (0..100):", (overall + 1) * 50)

# optionally exclude neutrals (sentiment == 0)
non_neutral = roberta_df["sentiment"] != 0
overall_non_neu = np.average(
    roberta_df.loc[non_neutral, "polarity"],
    weights=roberta_df.loc[non_neutral, "weight_likes"]
)
print("Weighted sentiment excl. neutral (−1..1):", overall_non_neu)
print("As % positive tilt excl. neutral:", (overall_non_neu + 1) * 50)

# quick breakdown to see which side drives the score
tmp = roberta_df.assign(weighted_contrib=roberta_df["polarity"] * roberta_df["weight_likes"])
print(
    tmp.groupby("label")
       .agg(total_weight=("weight_likes","sum"),
            contrib_sum=("weighted_contrib","sum"))
       .sort_values("contrib_sum", ascending=False)
)


roberta_df["polarity"] = roberta_df["confidence"] * roberta_df["weight_likes"] * roberta_df["sentiment"]
non_neutral = roberta_df["sentiment"] != 0

# print(roberta_sentiments)
# print(aggregate_label_conf(roberta_sentiments))

print(roberta_df.groupby("label").size())
# print(non_neutral.groupby("label").size())

pos = (roberta_df["sentiment"] > 0).sum()
neg = (roberta_df["sentiment"] < 0).sum()
neu = (roberta_df["sentiment"] == 0).sum()
total = len(roberta_df)

pprint({
    "pos%": float(pos/total)*100,
    "neg%": float(neg/total)*100,
    "neu%": float(neu/total)*100,
})
print("Excluding neutral comments")
pprint({
    "pos%": float(pos/(total-neu))*100,
    "neg%": float(neg/(total-neu))*100,
})
negative_comments = roberta_df[roberta_df["sentiment"] == "❌"]
positive_comments = roberta_df[roberta_df["sentiment"] == "✅"]
neutral_comments = roberta_df[roberta_df["sentiment"] == "😐"]
roberta_df.sort_values(by="likes", ascending=False, inplace=True)

# display(HTML(negative_comments.to_html(index=False)))
# display(HTML(positive_comments.to_html(index=False)))
# display(HTML(neutral_comments.to_html(index=False)))
display(HTML(roberta_df.to_html(index=False, max_rows=30)))
# display(HTML(roberta_df[non_neutral].to_html(index=False)))


In [ ]:
client = openai.OpenAI()
representation_model = OpenAI(client, model="gpt-4o", chat=True)
openai_BertTopic_model = BERTopic(representation_model=representation_model)

representation_model = KeyBERTInspired()
keybert_BertTopic_model = BERTopic(representation_model=representation_model)

In [ ]:
# BERTopic fitting
keybert_topics, keybert_probs = keybert_BertTopic_model.fit_transform(cleaned_tweets)
keybert_topic_info = keybert_BertTopic_model.get_topic_info()  # topic list with sizes
keybert_topic_info


openai_topics, openai_probs = openai_BertTopic_model.fit_transform(cleaned_tweets)
openai_topic_model = openai_BertTopic_model.reduce_topics(cleaned_tweets, nr_topics="auto")
openai_topic_info = openai_BertTopic_model.get_topic_info()  # topic list with sizes
openai_topic_info

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
vader_df = pd.DataFrame()
vader_df["comment"] = cleaned_comments
vader_sentiments = [analyzer.polarity_scores(sentence) for sentence in cleaned_comments]
# vader_df["neg"] = [sentiment["neg"] for sentiment in vader_sentiments]
# vader_df["neu"] = [sentiment["neu"] for sentiment in vader_sentiments]
# vader_df["pos"] = [sentiment["pos"] for sentiment in vader_sentiments]
# vader_df["compound"] = [sentiment["compound"] for sentiment in vader_sentiments]
vader_df["overall"] = [["❌", "😐", "✅"][np.argmax([sentiment["neg"], sentiment["neu"], sentiment["pos"]])] for sentiment in vader_sentiments]
vader_df["confidence"] = [sentiment[["neg", "neu", "pos"][np.argmax([sentiment["neg"], sentiment["neu"], sentiment["pos"]])]] for sentiment in vader_sentiments]
print(vader_df.groupby("overall").size())
vader_df

In [ ]:
from langchain_community.chat_models import ChatOllama

qwen3 = ChatOllama(model="qwen3:14b")
llama3_1 = ChatOllama(model="llama3.1:8b")
mistral = ChatOllama(model="mistral:7b")

In [ ]:
test_prompt = f"""
Perform sentiment analysis on the tweet and comments.  

- Give Overall sentiment: <SCORE from -1 to 1>.  

Input:  
Tweet: {stringify_tweet(top_20_tweets[0])}  
Comments: {json.dumps(cleaned_comments)}  
""".strip()

res = llama3_1.invoke(test_prompt)
print(res.content)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
futures = []
with ThreadPoolExecutor(max_workers=100) as executor:
    for comment in cleaned_comments:
        futures.append(executor.submit(llama3_1.invoke, f"Please give a sentiment analysis of the following comment: {comment}"))

for future in futures:
    res = future.result()
    print(res.content)
    print("-"*100)
